# VGG Face - 2

# Install TensorFlow-1.14 GPU.

In [0]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y
!pip uninstall tensorflow-gpu -y

# Install TensorFlow-1.14 and Keras-2.2.4.
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install --upgrade keras==2.2.4

# Restart the runtime.

# Import TensorFlow-1.14.

In [0]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)